#### **Lets build a database!**

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

#### **Lets first explore the different datasets and get a sense of what data we are working with**

In [29]:
file_names = ['game_log','park_codes','person_codes','team_codes']
df_dict = {}
for file in file_names:
    df_dict[file] = pd.read_csv(file+'.csv')

C:\Users\masv\AppData\Local\Temp\ipykernel_21276\2424821291.py:4: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dict[file] = pd.read_csv(file+'.csv')


In [30]:
df_dict['game_log']

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,NaN,2,RC1,NaN,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,NaN,3,CH1,NaN,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,NaN,2,TRO,NaN,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,...,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,NaN,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,...,Andres Blanco,6.0,altha001,Aaron Altherr,9.0,eickj001,Jerad Eickhoff,1.0,NaN,Y
171904,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,...,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,NaN,Y
171905,20161002,0,Sun,PIT,NL,162,SLN,NL,162,4,...,Jedd Gyorko,4.0,gricr001,Randal Grichuk,8.0,waina001,Adam Wainwright,1.0,NaN,Y


#### **Normalization of the tables**

Now, we need to normalize the tables and we need to create a primary key that interconnects the tables. There is no unique key in the table "game_logs" and needs to be created. This can be done in various ways, by e.g. consulting the data for viable columns or create custom columns. 

Luckily, Retrosheet have already built up a system for how to uniquely adress each game, and it is as follows:


**id**: Each game begins with a twelve character ID record which identifies the date, home team, and number of the game. For example, ATL198304080 should be read as follows. The first three characters identify the home team (the Braves). The next four are the year (1983). The next two are the month (April) using the standard numeric notation, 04, followed by the day (08). The last digit indicates if this is a single game (0), first game (1) or second game (2) if more than one game is played during a day, usually a double header The id record starts the description of a game thus ending the description of the preceding game in the file.



Here it can be seen that some of the older team names have only two letters and then a number. But lets treat the number as part of the name.

In [37]:
def create_retro_id(row):
    #row['date'] = pd.to_datetime(row['date'],format='%Y%m%d')
    id = row['h_name']+str(row['date'])+str(row['number_of_game'])
    return id

df_dict['game_log']['id'] = df_dict['game_log'].apply(create_retro_id,axis=1)
df_dict['game_log']['id']

0         FW1187105040
1         WS3187105050
2         RC1187105060
3         CH1187105080
4         TRO187105090
              ...     
171902    COL201610020
171903    PHI201610020
171904    SFN201610020
171905    SLN201610020
171906    WAS201610020
Name: id, Length: 171907, dtype: object

Lets create a database and insert the csv files into it!

In [38]:
conn = sqlite3.connect('test_database') 
c = conn.cursor()

for file in file_names:
    df_dict[file].to_sql(file,conn,if_exists='replace',index=False)
    conn.commit()

#conn.close()

In [17]:
#c.open()
r_df = pd.read_sql("select * from game_log",conn)
r_df

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,None,1,FW1,None,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,None,Y
1,18710505,0,Fri,BS1,None,1,WS3,None,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,None,2,RC1,None,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,None,Y
3,18710508,0,Mon,CL1,None,3,CH1,None,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,None,Y
4,18710509,0,Tue,BS1,None,2,TRO,None,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171902,20161002,0,Sun,MIL,NL,162,COL,NL,162,6,...,Pat Valaika,4.0,adamc001,Cristhian Adames,6.0,marqg001,German Marquez,1.0,None,Y
171903,20161002,0,Sun,NYN,NL,162,PHI,NL,162,2,...,Andres Blanco,6.0,altha001,Aaron Altherr,9.0,eickj001,Jerad Eickhoff,1.0,None,Y
171904,20161002,0,Sun,LAN,NL,162,SFN,NL,162,1,...,Joe Panik,4.0,gillc001,Conor Gillaspie,5.0,moorm003,Matt Moore,1.0,None,Y
171905,20161002,0,Sun,PIT,NL,162,SLN,NL,162,4,...,Jedd Gyorko,4.0,gricr001,Randal Grichuk,8.0,waina001,Adam Wainwright,1.0,None,Y
